# Data Collection - Web Scraping

In [3]:
import re
import requests
from bs4 import BeautifulSoup
import string

def extract_lyrics(url):

    try:
        page = requests.get(url).text
        soup = BeautifulSoup(page, "lxml")
        text = soup.find(class_="text separate")

        text_str = str(text).replace('<br/>','\n')
        text_str = re.sub('</*div.*>', '', text_str) # remove div tags
    except:
        text_str = None
       
    not_english = re.sub(f'[{string.ascii_letters}\W]+','',text_str)
    if (not not_english) or (len(not_english) < 20):
        # if the lyrics is in English skip it
        text_str = None

    
    return text_str

def get_song_info(track):
    
    base_url = 'http://www.website_name.ru'
    
    try:
        d = \
        {
            'artist':track.find(class_='st-artist').text,
            'title':track.find(class_='st-title').a.text,
            'lyrics_url':base_url+track.find(class_='st-title').a['href']
        }
    except Exception as e:
        d = None
    
    if d:
        lyrics = extract_lyrics(d['lyrics_url'])
        
        if lyrics:
            d['lyrics'] = lyrics
        else:
            d = None
    
    return d

In [41]:
urls = [f'http://www.website_name.ru/vysotskii.htm?page={page}' for page in range(1,4)]
urls

['http://www.website_name.ru/vysotskii.htm?page=1',
 'http://www.website_name.ru/vysotskii.htm?page=2',
 'http://www.website_name.ru/vysotskii.htm?page=3']

In [ ]:
%%time

song_urls = []

for url in urls:
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    for track_type in ['track', 'track playable']:
        tracks = soup.find(class_="songs-table").find_all(class_=track_type)
        for track in tracks:
            song_info = get_song_info(track)
            if song_info:
                song_urls.append(song_info)
                
                if not (len(song_urls)%10):
                    print(f"downloaded link #{len(song_urls):>3} : {song_info['lyrics_url']}")
                
print(f"# of downloaded song lyrics: {len(song_urls)}")

In [ ]:
for n, song_info in enumerate(song_urls):
    
    print(f"{n:>2}. {song_info['artist']} - {song_info['title']}  - {song_info['lyrics']}")

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=list(song_urls[0].keys()))

pd.set_option('max_colwidth',150)

idx = 0
for song_info in song_urls:
    df.loc[idx] = pd.Series(song_info)

    idx += 1
    
df.to_pickle("data/vysotskiy_lyrics_original.pkl")

df.head()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist      50 non-null     object
 1   title       50 non-null     object
 2   lyrics_url  50 non-null     object
 3   lyrics      50 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


# Data Cleaning 1st Stage - Unwanted Symbols and Words

In [1]:
import pandas as pd

df = pd.read_pickle("data/vysotskiy_lyrics_original.pkl")
df.reset_index(inplace=True, drop=True)
df.head()

,artist,title,lyrics
0,Высоцкий,Песня о друге,Если друг оказался вдруг\n И не друг и не враг...
1,Высоцкий,"""соглашайся хотя бы на рай в шалаше, если тере...","Здесь лапы у елей дрожат на весу,\nЗдесь птиц..."
2,Высоцкий,Парус,Парус \n\n\n\nА у дельфина взрезано брюхо винт...
3,Высоцкий,Корабли постоят,"Корабли постоят и ложатся на курс, \nНо они во..."
4,Высоцкий,Кассандра,ПЕСНЯ О ВЕЩЕЙ КАССАНДРЕ\nДолго Троя в положени...


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  50 non-null     object
 1   title   50 non-null     object
 2   lyrics  50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


### Add `id` column

In [3]:
import utils

df["id"] = ""
artist = 'Vysotskiy'.lower()

for idx in df.index:    
    id = f"{artist}_{utils.transliterate_ru_to_en(df.loc[idx, 'title'])}"

    if len(df[df.id == id]) > 0:
        df.loc[idx, 'id'] = id+'_'+str(len(df[df.id == id]))
    else:
        df.loc[idx, 'id'] = id
    
df.head()

,artist,title,lyrics,id
0,Высоцкий,Песня о друге,Если друг оказался вдруг\n И не друг и не враг...,vysotskiy_pesnja_o_druge
1,Высоцкий,"""соглашайся хотя бы на рай в шалаше, если тере...","Здесь лапы у елей дрожат на весу,\nЗдесь птиц...",vysotskiy_soglashajsja_hotja_by_na_raj_v_shala...
2,Высоцкий,Парус,Парус \n\n\n\nА у дельфина взрезано брюхо винт...,vysotskiy_parus
3,Высоцкий,Корабли постоят,"Корабли постоят и ложатся на курс, \nНо они во...",vysotskiy_korabli_postojat
4,Высоцкий,Кассандра,ПЕСНЯ О ВЕЩЕЙ КАССАНДРЕ\nДолго Троя в положени...,vysotskiy_kassandra


In [4]:
for idx in df.index:
    
    song_info = df.loc[idx]
    print(f"\n\n{idx:>5}({song_info['id']}) {song_info['artist']} - {song_info['title']}\n\n  - {song_info['lyrics']}")



    0(vysotskiy_pesnja_o_druge) Высоцкий - Песня о друге

  - Если друг оказался вдруг
 И не друг и не враг, а – так..
 Если сразу не разберёшь,
 Плох он или хорош -

Парня в горы тяни – рискни,
 Не бросай одного его,
 Пусть он в связке с тобой в одной -
 Там поймешь, кто такой

Если парень в горах – не ах,
 Если сразу раскис – и вниз,
 Шаг ступил на ледник – и сник,
 Оступился – и в крик,

Значит, рядом с тобой чужой,
 Ты его не брани – гони:
 Вверх таких не берут и тут
 Про таких не поют

Если ж он не скулил, не ныл,
 Пусть он хмур был и зол, но шёл,
 А когда ты упал со скал,
 Он стонал, но держал,

Если шёл за тобой, как в бой,
 На вершине стоял, хмельной,
 Значит – как на себя самого,
 Положись на него.



    1(vysotskiy_soglashajsja_hotja_by_na_raj_v_shalashe_esli_terem_uzhe_kto_to_zanjal) Высоцкий - "соглашайся хотя бы на рай в шалаше, если терем уже кто то занял..."

  - Здесь лапы у елей  дрожат на весу,
Здесь птицы щебечут тревожно,
Живешь, в заколдованном диком лесу,
От ку

### Clean Lyrics From Chords and Other Non-Lyrics Symbols

We need to remove the row:
- if 'вступление' is the only cyrillic word in the row
- if 'припев:' is the only word in the row
- if 'проигрыш:' is the only word in the row
- if 'соло:' is the only word in the row
- if it is a string like 'припев 2 раза'

In [7]:
from tqdm import tqdm
import utils

for idx in tqdm(df.index):
    
    song_info = df.loc[idx]
    text, status   = utils.clean_text(df.loc[idx, 'lyrics'])
    
    if status > 0:
        # error occured
        break
        
    df.loc[idx, 'lyrics'] = text
    
    #print(f"{idx:>5}. {song_info['artist']} - {song_info['title']}\n\n  {df.loc[idx, 'lyrics']}")
    
df.to_pickle("data/vysotskiy_lyrics_clean_first_stage.pkl")

100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 1253.29it/s]


# Data Cleaning 2nd Stage - Drop duplicates

In [8]:
import pandas as pd

df_clean = pd.read_pickle("data/vysotskiy_lyrics_clean_first_stage.pkl")
df_clean.reset_index(inplace=True, drop=True)
print(df_clean.info())
df_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  50 non-null     object
 1   title   50 non-null     object
 2   lyrics  50 non-null     object
 3   id      50 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB
None


,artist,title,lyrics,id
0,Высоцкий,Песня о друге,если друг оказался вдруг\n и не друг и не враг...,vysotskiy_pesnja_o_druge
1,Высоцкий,"""соглашайся хотя бы на рай в шалаше, если тере...","здесь лапы у елей дрожат на весу,\nздесь птиц...",vysotskiy_soglashajsja_hotja_by_na_raj_v_shala...
2,Высоцкий,Парус,парус \nа у дельфина взрезано брюхо винтом! \n...,vysotskiy_parus
3,Высоцкий,Корабли постоят,"корабли постоят и ложатся на курс, \nно они во...",vysotskiy_korabli_postojat
4,Высоцкий,Кассандра,песня о вещей кассандре\nдолго троя в положени...,vysotskiy_kassandra


In [9]:
kept_song_ids=[]

removed_songs_ids = []

pre_kept_song_ids     = [*df_clean[df_clean.id.isin(kept_song_ids)].index]
pre_removed_songs_ids = [*df_clean[df_clean.id.isin(removed_songs_ids)].index]
processed_song_ids = sorted(pre_kept_song_ids+pre_removed_songs_ids)
processed_song_ids[-10:]

[]

In [10]:
import re
# show the list of duplicates

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

corpus = [re.sub('\n',' ', txt) for txt in [*df_clean.lyrics.values]]
vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform(corpus)
pairwise_similarity = tfidf * tfidf.T
pairwise_similarity = pairwise_similarity.toarray()

duplicates = []
for idx1 in tqdm(df_clean.index):
        
    if idx1 in processed_song_ids:
        continue
        
    if idx1 in set(sorted([item for sublist in [(ix1, ix2) for ix1, ix2, _  in duplicates] for item in sublist])):
        continue
    
    # exclude songs comparison to themselves
    pairwise_similarity[idx1, idx1] = 0
    
    similarities = [(idx, sim) for idx, sim in enumerate(pairwise_similarity[idx1]) if sim > 0.5]
    
    for idx2, sim in similarities:
        title1 = df_clean.loc[idx1, 'title']
        title2 = df_clean.loc[idx2, 'title']
        
        duplicates.append((idx1, idx2, sim))
        print(f"{title1}({idx1})  - {title2}({idx2}): {sim}")
        

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 25046.60it/s]

Парус(2)  - Парус(37): 0.9999999999999996
Корабли постоят(3)  - Корабли 2(19): 0.9651769243290972
Девушка из Нагасаки(5)  - Девушка из Нагасаки(38): 0.9999999999999996
Большой Каретный(8)  - Большой Каретный(39): 0.9999999999999998
Случай в ресторане(9)  - Случай в ресторане(40): 0.999999999999999
Песня о госпитале(11)  - Песня о госпитале(41): 0.9999999999999997
Братские могилы(15)  - Братские могилы(42): 0.9999999999999997
Песня командировочного(16)  - Песня командировочного(43): 0.9999999999999997
Песня летчика(18)  - песня о бое в воздухе(29): 0.9547213536570663
Песня летчика(18)  - Песня летчика(44): 0.9999999999999999
Мы вращаем Землю(20)  - Мы вращаем Землю(45): 1.0
Я женщин не бил до семнадцати(22)  - Я женщин не бил до семнадцати(46): 0.9999999999999996
Я в деле(24)  - Я в деле(47): 0.9999999999999997
За меня невеста(25)  - За меня невеста(48): 0.9999999999999998
Эй, шофер(30)  - Эй, шофер(49): 0.9999999999999993


In [11]:
import string

def lyrics_len(text):
    text  = re.sub(r'['+string.punctuation+']+',' ',text)
    return len(text.split())

In [12]:
from collections import Counter
lyrics_lens = [lyrics_len(df_clean.loc[ix, 'lyrics']) for ix in df_clean.index]
d = Counter(sorted(lyrics_lens))

[*d.keys()], [*d.values()]
lst = [l for l in d.keys() if l > 20]
print(f"average lyrics word-length: {sum(lst)/len(lst):.1f}")

average lyrics word-length: 172.9


In [13]:
[(ix, df_clean.loc[ix, 'title']) for ix in df_clean.index if lyrics_len(df_clean.loc[ix, 'lyrics']) < 20]

[]

In [14]:
import string

def lyrics_len(text):
    text  = re.sub(r'['+string.punctuation+']+',' ',text)
    return len(text.split())


kept_song_ids = [] # kepp only the first version among all duplicates

current_song_idx = -1
current_song_dup_ids = []
for idx1, idx2, similarity in duplicates:

    if current_song_idx != idx1:
        
        if current_song_dup_ids:
            # choose kept idx
            # if the first song has len(words) < 100
            # choose the song with max number of words
            
            if lyrics_len(df_clean.loc[current_song_dup_ids[0], 'lyrics']) < 100:
                lyrics_lens = [lyrics_len(df_clean.loc[ix, 'lyrics']) for ix in current_song_dup_ids]
                max_idx = np.argmax(lyrics_lens)
                if lyrics_lens[max_idx] > 20:
                    # if the max lyrics length is less than 20 do not include any of the songs
                    kept_song_ids.append(current_song_dup_ids[max_idx])
            else:
                kept_song_ids.append(current_song_dup_ids[0])

        current_song_dup_ids = []
        current_song_idx = idx1
        current_song_dup_ids.append(idx1)
        
    
    current_song_dup_ids.append(idx2)
    
    lyrics1 = df_clean.loc[idx1, 'lyrics'].strip('\n').split('\n')
    lyrics2 = df_clean.loc[idx2, 'lyrics'].strip('\n').split('\n')
    comparison = zip(lyrics1, lyrics2)
    # put id of a song that you want to keep, all other songs in duplicates will be dropped

    str_left  = df_clean.loc[idx1, 'title']+'('+str(idx1)+')'
    str_right = df_clean.loc[idx2, 'title']+'('+str(idx2)+')'

    print(f"\n\n{str_left:>50}  =   {str_right} : similarity: {similarity:.5f}\n")

    for row1, row2 in comparison:

        print(f"{row1:>50}  =   {row2}")
        
    print()
    print(' '*10+'-'*100)
    
            
if lyrics_len(df_clean.loc[current_song_dup_ids[0], 'lyrics']) < 100:
    lyrics_lens = [lyrics_len(df_clean.loc[ix, 'lyrics']) for ix in current_song_dup_ids]
    max_idx = np.argmax(lyrics_lens)
    kept_song_ids.append(current_song_dup_ids[max_idx])
else:
    kept_song_ids.append(current_song_dup_ids[0])




                                          Парус(2)  =   Парус(37) : similarity: 1.00000

                                            парус   =   парус 
              а у дельфина взрезано брюхо винтом!   =   а у дельфина взрезано брюхо винтом! 
               выстрела в спину не ожидает никто.   =    выстрела в спину не ожидает никто. 
                    на батарее нету снарядов уже.   =    на батарее нету снарядов уже. 
                          надо быстрее на вираже!   =    надо быстрее на вираже! 
                           парус! порвали, парус!   =    парус! порвали, парус! 
                             каюсь! каюсь! каюсь!   =    каюсь! каюсь! каюсь! 
         даже в дозоре можешь не встретить врага.   =    даже в дозоре можешь не встретить врага. 
                   это не горе - если болит нога.   =    это не горе - если болит нога. 
        петли дверные многим скрипят, многим поют   =    петли дверные многим скрипят, многим поют 
                  кто вы такие вас здесь н

In [15]:
len(kept_song_ids), kept_song_ids

(14, [2, 19, 5, 8, 9, 11, 15, 16, 18, 20, 22, 24, 25, 30])

In [16]:
#kept_song_ids = []

removed_songs = set(sorted([item for sublist in [(ix1, ix2) for ix1, ix2, _  in duplicates] for item in sublist]))
removed_songs = sorted(removed_songs.difference(set(kept_song_ids)))

kept_song_ids = [*set(pre_kept_song_ids).union(set(kept_song_ids))]
removed_songs = [*set(pre_removed_songs_ids).union(set(removed_songs))]

kept_song_ids     = [*df_clean[df_clean.index.isin(kept_song_ids)].id.values]
removed_songs_ids = [*df_clean[df_clean.index.isin(removed_songs)].id.values]
kept_song_ids

['vysotskiy_parus',
 'vysotskiy_devushka_iz_nagasaki',
 'vysotskiy_bolshoj_karetnyj',
 'vysotskiy_sluchaj_v_restorane',
 'vysotskiy_pesnja_o_gospitale',
 'vysotskiy_bratskie_mogily',
 'vysotskiy_pesnja_komandirovochnogo',
 'vysotskiy_pesnja_letchika',
 'vysotskiy_korabli_2',
 'vysotskiy_my_vraschaem_zemlju',
 'vysotskiy_ja_zhenschin_ne_bil_do_semnadtsati',
 'vysotskiy_ja_v_dele',
 'vysotskiy_za_menja_nevesta',
 'vysotskiy_ej_shofer']

In [17]:
removed_songs_ids

['vysotskiy_korabli_postojat',
 'vysotskiy_pesnja_o_boe_v_vozduhe',
 'vysotskiy_parus_1',
 'vysotskiy_devushka_iz_nagasaki_1',
 'vysotskiy_bolshoj_karetnyj_1',
 'vysotskiy_sluchaj_v_restorane_1',
 'vysotskiy_pesnja_o_gospitale_1',
 'vysotskiy_bratskie_mogily_1',
 'vysotskiy_pesnja_komandirovochnogo_1',
 'vysotskiy_pesnja_letchika_1',
 'vysotskiy_my_vraschaem_zemlju_1',
 'vysotskiy_ja_zhenschin_ne_bil_do_semnadtsati_1',
 'vysotskiy_ja_v_dele_1',
 'vysotskiy_za_menja_nevesta_1',
 'vysotskiy_ej_shofer_1']

In [18]:
# drop duplicates

df_clean = df_clean[~df_clean.id.isin(removed_songs_ids)]
df_clean.reset_index(inplace=True, drop=True)
df_clean.to_pickle("data/vysotskiy_lyrics_clean_second_stage.pkl")

print(df_clean.info())
df_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  35 non-null     object
 1   title   35 non-null     object
 2   lyrics  35 non-null     object
 3   id      35 non-null     object
dtypes: object(4)
memory usage: 1.2+ KB
None


,artist,title,lyrics,id
0,Высоцкий,Песня о друге,если друг оказался вдруг\n и не друг и не враг...,vysotskiy_pesnja_o_druge
1,Высоцкий,"""соглашайся хотя бы на рай в шалаше, если тере...","здесь лапы у елей дрожат на весу,\nздесь птиц...",vysotskiy_soglashajsja_hotja_by_na_raj_v_shala...
2,Высоцкий,Парус,парус \nа у дельфина взрезано брюхо винтом! \n...,vysotskiy_parus
3,Высоцкий,Кассандра,песня о вещей кассандре\nдолго троя в положени...,vysotskiy_kassandra
4,Высоцкий,Девушка из Нагасаки,он капитан и родина его марсель\nон любит спор...,vysotskiy_devushka_iz_nagasaki


# Data Cleaning 3rd Stage -  Remove and Replace Incorrect Words

In [19]:
import pandas as pd

df_clean = pd.read_pickle("data/vysotskiy_lyrics_clean_second_stage.pkl")
df_clean.reset_index(inplace=True, drop=True)
print(df_clean.info())
df_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  35 non-null     object
 1   title   35 non-null     object
 2   lyrics  35 non-null     object
 3   id      35 non-null     object
dtypes: object(4)
memory usage: 1.2+ KB
None


,artist,title,lyrics,id
0,Высоцкий,Песня о друге,если друг оказался вдруг\n и не друг и не враг...,vysotskiy_pesnja_o_druge
1,Высоцкий,"""соглашайся хотя бы на рай в шалаше, если тере...","здесь лапы у елей дрожат на весу,\nздесь птиц...",vysotskiy_soglashajsja_hotja_by_na_raj_v_shala...
2,Высоцкий,Парус,парус \nа у дельфина взрезано брюхо винтом! \n...,vysotskiy_parus
3,Высоцкий,Кассандра,песня о вещей кассандре\nдолго троя в положени...,vysotskiy_kassandra
4,Высоцкий,Девушка из Нагасаки,он капитан и родина его марсель\nон любит спор...,vysotskiy_devushka_iz_nagasaki


### Small fix - remove songs

In [20]:
'''df = df_clean[df_clean.title=='Запись с концерта в ДК Меридиан 28 августа 1998 Москва']
if len(df)>0:
    for idx in df.index:
        df_clean.drop(index=[idx], inplace=True)
        
        
df = df_clean[df_clean.title == "А мы танцуем на палубе тонущего корабля"]

if len(df)>0:
    for idx in df.index:
        df_clean.drop(index=[idx], inplace=True)

    
df_clean.reset_index(inplace=True, drop=True)'''

'df = df_clean[df_clean.title==\'Запись с концерта в ДК Меридиан 28 августа 1998 Москва\']\nif len(df)>0:\n    for idx in df.index:\n        df_clean.drop(index=[idx], inplace=True)\n        \n        \ndf = df_clean[df_clean.title == "А мы танцуем на палубе тонущего корабля"]\n\nif len(df)>0:\n    for idx in df.index:\n        df_clean.drop(index=[idx], inplace=True)\n\n    \ndf_clean.reset_index(inplace=True, drop=True)'

In [21]:
import re

all_words = sorted(set(re.sub('\n',' ',' '.join([*df_clean.lyrics.values])).split()))
print(f"total number of words : {len(all_words)}")
sorted(all_words)

total number of words : 2818


['"',
 '"в',
 '"впишите',
 '"да',
 '"дорогие!"',
 '"друзья!"',
 '"дух',
 '"заколотый',
 '"казбек"',
 '"как',
 '"капитан,',
 '"когда',
 '"кто',
 '"марсельезу"',
 '"мать...",',
 '"машка,',
 '"мессер",',
 '"милый!',
 '"милый"',
 '"морфушу"',
 '"нет,',
 '"первый",',
 '"пират"',
 '"поверьте,',
 '"проше,',
 '"ребята!',
 '"стойте!',
 '"таганку"',
 '"таганку",-',
 '"так',
 '"три',
 '"ту",',
 '"ты".',
 '"тэже"',
 '"узнать',
 '"улетел,',
 '"флибустьеры".',
 '"хоть',
 '"чаевые"',
 '"это',
 '"я',
 '"ясно',
 '(1.4',
 ',',
 ',-',
 '-',
 '.',
 '...вот',
 '07,',
 '07.',
 '1963',
 '1963.',
 '1966',
 '1968',
 '6',
 'am',
 'dm',
 '«дышу»',
 '«живу».',
 '«я',
 'а',
 'а,',
 'агоний,',
 'азиаты',
 'академик',
 'аккредитив',
 'алтаря!',
 'алые',
 'альпинистка',
 'анашу...',
 'ангел',
 'ангел,',
 'ангел-ас,',
 'ангелы',
 'ангельский',
 'антарктикой.',
 'апоморфин!',
 'арбате,',
 'арктика',
 'архангел',
 'аскет!',
 'асхат.',
 'атаку',
 'афон!',
 'ах',
 'ах,',
 'ашхабада,',
 'б',
 'баба',
 'бабами-ягами,',
 'ба

In [22]:
removed_words = []

In [23]:
from tqdm import tqdm
import re

stop = False
for idx in tqdm(df_clean.index):
        
    for word in removed_words:
        try:
            df_clean.loc[idx, 'lyrics']   = re.sub(word, '', df_clean.loc[idx, 'lyrics'])
        except Exception as e:
            print(e)
            print(idx)
            print(word)
            stop = True
            break
            
    if stop:
        break
    
df_clean.to_pickle("data/vysotskiy_lyrics.pkl")

100%|███████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 35052.68it/s]
